<img src='Cover5-01.png'>

# Анализ тональности текста (Sentiment Analysis)

**Sentiment analysis (по-русски, анализ тональности)** — это область компьютерной лингвистики, которая занимается изучением мнений и эмоций в текстовых документах. 

Анализ тональности находит свое практическое применение в разных областях:
- социология — собираем данные из соц. сетей (например, о религиозных взглядах)
- политология — собираем данные из блогов о политических взглядах населения
- маркетинг — анализируем Твиттер, чтобы узнать какая модель ноутбуков пользуется наибольшим спросом
- медицина и психология — определяем депрессию у пользователей соц. сетей

## Подходы к классификации тональности

Анализ тональности обычно определяют как одну из задач компьютерной лингвистики, т.е. подразумевается, что мы можем найти и классифицировать тональность, используя инструменты обработки естественного языка (такие как теггеры, парсеры и др.). Сделав большое обобщение, можно разделить существующие подходы на следующие категории:

<img src='t4.png', align="left", width="600">


**Первый тип систем** состоит из набора правил, применяя которые система делает заключение о тональности текста. Например, для предложения «Я люблю кофе», можно применить следующее правило:

- если сказуемое ("люблю") входит в положительный набор глаголов ("люблю", "обожаю", "одобряю" ...) и в предложении не имеется отрицаний, то классифицировать тональность как "положительная"

Многие коммерческие системы используют данный подход, несмотря на то что он требует больших затрат, т.к. для хорошей работы системы необходимо составить большое количество правил.

Подходы, **основанные на словарях**, используют так называемые тональные словари (affective lexicons) для анализа текста. В простом виде тональный словарь представляет из себя список слов со значением тональности для каждого слова. Вот пример из базы ANEW, переведенный на русский:

слово валентность --- (1-9)
- счастливый ---	8.21
- хороший	--- 7.47
- скучный	--- 2.95
- сердитый --- 2.85
- грустный --- 1.61


**Машинное обучение без учителя** представляет собой, наверное, наиболее интересный и в то же время наименее точный метод анализа тональности. Одним из примеров данного метода может быть автоматическая кластеризация документов.

**Машинное обучение с учителем** является наиболее распространенным методом, используемым в исследованиях. Его суть состоит в том, чтобы обучить машинный классификатор на коллекции заранее размеченных текстах, а затем использовать полученную модель для анализа новых документов. Именно про этот метод мы и поговорим далее.

## Задача

Сделать классификатор тональности коротких текстовых сообщений (твитов). Алгоритм должен уметь классифицировать сообщения на два класса: сообщения с положительной эмоциональной окраской и сообщения с отрицательной окраской.

## Данные

- Приблизительно из 225 тысяч размеченных твитов 
- Для разметки на два класса (положительные и отрицательные)

In [1]:
import pandas as pd

In [2]:
p_data = pd.read_csv('data/positive.csv', sep=';', header=None)
n_data = pd.read_csv('data/negative.csv', sep=';', header=None)

In [3]:
dataset = pd.concat([p_data, n_data])

In [4]:
dataset = dataset[[3, 4]]
dataset.columns = ['text', 'label']

## Морфолгоический анализатор и предварительная обработка

**Морфологический анализатор** для русского языка — программа, которая приводит слово к начальной форме, определяет падеж, находит словоформы. Этот процесс называется **лемматизация**.

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [6]:
def text_cleaner(text):
    # к нижнему регистру
    text = text.lower()
    
    # оставляем в предложении только русские буквы (таким образом
    # удалим и ссылки, и имена пользователей, и пунктуацию и т.д.)
    alph = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    
    cleaned_text = ''
    for char in text:
        if (char.isalpha() and char[0] in alph) or (char == ' '):
            cleaned_text += char
        
    result = []
    for word in cleaned_text.split():
        # лемматизируем
        result.append(morph.parse(word)[0].normal_form)
                              
    return ' '.join(result)

dataset['text'] = dataset['text'].apply(text_cleaner)

dataset.to_csv('data/cleaned_data.csv')

Отлично, теперь у нас есть очищенный и подготовленный текст!
Теперь требуется построить признаковое описание текста. Для этого мы будем использовать такие инструменты:

- CountVectorizer
- TfidfVectorizer 
    
###  CountVectorizer
    
CountVectorizer преобразовывает входной текст в матрицу, значениями которой, являются количества вхождения данного ключа(слова) в текст. Рассмотрим следующий пример.

Допустим есть таблица с текстовыми значениями: 

_________________________

раз два три

три четыре два два

раз раз раз четыре
_________________________



Для начала CountVectorizer собирает уникальные ключи из всех записей, в нашем примере это будет:

[раз, два, три, четыре]

Длина списка из уникальных ключей и будет длиной нашего закодированного текста (в нашем случае это 4). А номера элементов будут соответствовать, количеству раз встречи данного ключа с данным номером в строке:

раз два три --> [1,1,1,0]

три четыре два два --> [0,2,1,1]

Соответственно после кодировки, применения данного метода мы получим:

_________________________

1,1,1,0

0,2,1,1

3,0,0,1
_________________________


### TfidfVectorizer

TfidfVectorizer действует похожим образом, но вместо количества вхождения слова в текст считает TF-IDF.

**TF-IDF** (от англ. TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален количеству употребления этого слова в документе, и обратно пропорционален частоте употребления слова в других документах коллекции.

**TF** (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова 
$t_{{i}}$ в пределах отдельного документа.


$TF(t,d)=\frac {n_{t}}{\sum _{k}n_{k}}$


где $n_t$ есть число вхождений слова 
$t$ в документ, а в знаменателе — общее число слов в данном документе.

IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции. Основоположником данной концепции является Карен Спарк Джонс[1]. Учёт IDF уменьшает вес широкоупотребительных слов. Для каждого уникального слова в пределах конкретной коллекции документов существует только одно значение IDF.

$IDF(t,D)=\log {\frac {|D|}{|\{\,d_{i}\in D\mid t\in d_{i}\,\}|}}$

где

$|D|$ — число документов в коллекции;

$|\{\,d_{i}\in D\mid t\in d_{i}\,\}|$ — число документов из коллекции $D$, в которых встречается 
$t$ (когда $n_{t}\neq 0$).

Выбор основания логарифма в формуле не имеет значения, поскольку изменение основания приводит к изменению веса каждого слова на постоянный множитель, что не влияет на соотношение весов.

Таким образом, мера TF-IDF является произведением двух сомножителей: $TF\times IDF$

Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах.

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# считываем подготовленный датасет
dataset = pd.read_csv('data/cleaned_data.csv', index_col=0).dropna()

In [9]:
# n-граммная схема, которая будут использоваться в работе
# например, (1, 3) означает униграммы + биграммы + триграммы
ngram_scheme = (1, 2)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
print('N-gram Scheme:', ngram_scheme)

count_vectorizer = CountVectorizer(analyzer = "word", ngram_range=ngram_scheme) 
tfidf_vectorizer = TfidfVectorizer(analyzer = "word", ngram_range=ngram_scheme)


N-gram Scheme: (1, 2)


In [13]:
import sklearn
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit, cross_val_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

/Users/flaren/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/flaren/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Теперь запустим классификатор

In [14]:
vectorizer = count_vectorizer

X = vectorizer.fit_transform(dataset['text'])
y = dataset['label']

cv = ShuffleSplit(len(y), n_iter=5, test_size=0.3, random_state=0)

# наивный байес
clf = MultinomialNB()
NB_result = cross_val_score(clf, X, y, cv=cv).mean()

# линейный классификатор
clf = SGDClassifier()

parameters = {
    'loss': ('log', 'hinge'),
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'alpha': [0.001, 0.0001, 0.00001, 0.000001]
}

gs_clf = GridSearchCV(clf, parameters, cv=cv, n_jobs=-1)
gs_clf = gs_clf.fit(X, y)

L_result = gs_clf.best_score_

print('NB:', NB_result.mean())
print('Linear:', L_result)
print('Linear Parameters:', gs_clf.best_params_)
print()

NB: 0.7364046495907359
Linear: 0.7414921600611324
Linear Parameters: {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2'}



## Задание

- Используя предложенные выше инструменты, попытаться повысить качество классификации
- Верно ли, что чем больше n в n-граммных схемах, тем точнее работает алгоритм? Почему?

Ссылки на источники:

- https://habr.com/post/49421/
- https://habr.com/post/205360/
- Ю. В. Рубцова. Построение корпуса текстов для настройки тонового классификатора // Программные продукты и системы, 2015, №1(109), –С.72-78